In [9]:
#pip install nltk (in case you do not have nltk)
import nltk
import json
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isaac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
#We load the products info in an array
products_path = '../../data/fashion_products_dataset.json'
with open(products_path, "r", encoding="utf-8") as f:
    products = json.load(f)

In [7]:
#We create a function to pre-process a line
def build_terms(line):
    """
    Preprocess a line:
    ●  Removing stop words 
    ●  Tokenization 
    ●  Removing punctuation marks 
    ●  Stemming 
    ●  Transforming to lowercase

    Argument:
    line -- string (text) to be preprocessed

    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    line = line.lower()
    line = line.split()
    line = [x for x in line if x not in stop_words]
    line = [stemmer.stem(word) for word in line]
    return line

In [14]:
print(products[0])

{'_id': 'fa8e22d6-c0b6-5229-bb9e-ad52eda39a0a', 'actual_price': '2,999', 'average_rating': '3.9', 'brand': 'York', 'category': 'Clothing and Accessories', 'crawled_at': 1612987911000, 'description': 'Yorker trackpants made from 100% rich combed cotton giving it a rich look.Designed for Comfort,Skin friendly fabric,itch-free waistband & great for all year round use Proudly made in India', 'discount': '69% off', 'images': ['https://rukminim1.flixcart.com/image/128/128/jr3t5e80/track-pant/z/y/n/m-1005combo2-yorker-original-imafczg3xfh5qqd4.jpeg?q=70', 'https://rukminim1.flixcart.com/image/128/128/jr58l8w0/track-pant/w/d/a/l-1005combo8-yorker-original-imafczg3pgtxgraq.jpeg?q=70'], 'out_of_stock': False, 'pid': 'TKPFCZ9EA7H5FYZH', 'product_details': [{'Style Code': '1005COMBO2'}, {'Closure': 'Elastic'}, {'Pockets': 'Side Pockets'}, {'Fabric': 'Cotton Blend'}, {'Pattern': 'Solid'}, {'Color': 'Multicolor'}], 'seller': 'Shyam Enterprises', 'selling_price': '921', 'sub_category': 'Bottomwear', 

In [ ]:
#We preprocess all titles and descriptions of the products


In [16]:
def information(products):
    new_products = []
    for product in products:
        elements = ["_id", "title", "description", "brand", "category", "sub_category", "product_details", "seller", "out_of_stock", "selling_price", "discount", "actual_price", "average_rating", "url"]
        process_product = {}
        for element in elements:
            process_product[element] = product[element]
        new_products.append(process_product)
    return new_products

In [ ]:
products = information(products)
for line in products:
    line["title"] = build_terms(line["title"])
    line["description"] = build_terms(line["description"])

{'_id': 'fa8e22d6-c0b6-5229-bb9e-ad52eda39a0a', 'title': 'Solid Women Multicolor Track Pants', 'description': 'Yorker trackpants made from 100% rich combed cotton giving it a rich look.Designed for Comfort,Skin friendly fabric,itch-free waistband & great for all year round use Proudly made in India', 'brand': 'York', 'category': 'Clothing and Accessories', 'sub_category': 'Bottomwear', 'product_details': [{'Style Code': '1005COMBO2'}, {'Closure': 'Elastic'}, {'Pockets': 'Side Pockets'}, {'Fabric': 'Cotton Blend'}, {'Pattern': 'Solid'}, {'Color': 'Multicolor'}], 'seller': 'Shyam Enterprises', 'out_of_stock': False, 'selling_price': '921', 'discount': '69% off', 'actual_price': '2,999', 'average_rating': '3.9', 'url': 'https://www.flipkart.com/yorker-solid-men-multicolor-track-pants/p/itmd2c76aadce459?pid=TKPFCZ9EA7H5FYZH&lid=LSTTKPFCZ9EA7H5FYZHVYXWP0&marketplace=FLIPKART&srno=b_1_1&otracker=browse&fm=organic&iid=177a46eb-d053-4732-b3de-fcad6ff59cbd.TKPFCZ9EA7H5FYZH.SEARCH&ssid=utkd4t3gb